In [1]:
import torch
from pathlib import Path
import os
from ultralytics import YOLO
from ultralytics.models.yolo.model import DetectionModel # Ensure 

In [ ]:
# --- 1. Configuration ---
CONFIG_NAME = 'yolo11s-dspan.yaml' # Name of the YAML file for the modified LCBHAM model
# File Paths
current_dir = os.getcwd()
target_yaml_path = os.path.join(current_dir, 'ultralytics', 'cfg', 'models', '11', CONFIG_NAME)
source_weights_path = os.path.join(current_dir, 'yolo11s.pt')           # Pre-trained standard YOLOv11s
output_weights_path = os.path.join(current_dir, 'runs', 'models', f'{CONFIG_NAME.split(".")[0]}.pt') # Output path for transferred weights

# Data Paths
data_path = os.path.join(current_dir, 'datasets', 'AblationDataset', 'flow', 'flow.yaml')
test_data_path = os.path.join(current_dir, 'datasets', 'TestDataset', 'flowtest', 'flowtest.yaml')
test_data_path_wt = os.path.join(current_dir, 'datasets', 'TestDataset', 'watertrashtest', 'watertrashtest.yaml')

# Model Parameters (Ensure these match your dataset and model)
nc = 1 # Number of classes (adjust if your dataset differs from COCO)

# LCBHAM Layer Indices (Confirm these match your target_yaml_path)
lcbham_layers_indices = {17}

In [ ]:
# --- 2. Perform Weight Transfer (if necessary) ---

if not os.path.exists(output_weights_path):
    print(f"Transfer weights file '{output_weights_path}' not found. Performing weight transfer...")

    # --- Load Source Weights ---
    print(f"Loading source weights from {source_weights_path}...")
    try:
        source_ckpt = torch.load(source_weights_path, map_location=torch.device('cpu'), weights_only=False)

        if hasattr(source_ckpt.get('model'), 'state_dict'):
            source_state_dict = source_ckpt['model'].float().state_dict()
        elif isinstance(source_ckpt.get('model'), dict):
             source_state_dict = source_ckpt['model']
        elif isinstance(source_ckpt, dict) and not 'model' in source_ckpt:
             source_state_dict = source_ckpt
        else:
            print("Available keys in source_ckpt:", source_ckpt.keys() if isinstance(source_ckpt, dict) else "Not a dict")
            raise ValueError("Could not extract state_dict from source checkpoint.")
        print(f"Source weights loaded and state_dict extracted.")
    except Exception as e:
        print(f"Error loading source weights: {e}")
        exit()

    # --- Build Target Model Structure ---
    print(f"Building target model structure from {target_yaml_path}...")
    try:
        # Ensure your custom LCBHAM module is defined/imported before this line
        target_model = DetectionModel(cfg=str(target_yaml_path), ch=3, nc=nc)
    except Exception as e:
         print(f"Error building target model: {e}")
         print("Ensure LCBHAM is defined, the YAML path is correct, and nc={nc} is appropriate.")
         exit()
    target_state_dict = target_model.state_dict()
    print(f"Target model structure built.")

    # --- Weight Transfer Logic ---
    print("Starting weight transfer...")
    new_state_dict = {}
    transferred_count = 0
    skipped_count = 0
    lcbham_conv_transferred = set()
    lcbham_bn_transferred = set()

    for k_target, v_target in target_state_dict.items():
        k_source = k_target
        layer_index_str = k_target.split('.')[1] # Get layer index string like '17'

        if layer_index_str.isdigit() and int(layer_index_str) in lcbham_layers_indices:
            layer_index = int(layer_index_str)
            if f".{layer_index_str}.conv_block.0." in k_target: # Conv part of LCBHAM
                k_source = k_target.replace("conv_block.0.", "conv.")
                lcbham_conv_transferred.add(layer_index)
            elif f".{layer_index_str}.conv_block.1." in k_target: # BN part of LCBHAM
                k_source = k_target.replace("conv_block.1.", "bn.")
                lcbham_bn_transferred.add(layer_index)

        if k_source in source_state_dict and source_state_dict[k_source].shape == v_target.shape:
            new_state_dict[k_target] = source_state_dict[k_source]
            transferred_count += 1
        else:
            new_state_dict[k_target] = v_target
            skipped_count += 1

    print("\n--- Transfer Summary ---")
    print(f"Total keys in target model: {len(target_state_dict)}")
    print(f"Weights transferred: {transferred_count}")
    print(f"Weights skipped/kept from target: {skipped_count}")
    for idx in lcbham_layers_indices:
        if idx in lcbham_conv_transferred: print(f"Successfully mapped Conv weights for LCBHAM layer {idx}.")
        if idx in lcbham_bn_transferred: print(f"Successfully mapped BN weights for LCBHAM layer {idx}.")

    # --- Load New State Dict and Save Checkpoint ---
    target_model.load_state_dict(new_state_dict, strict=False)
    print("\nLoaded transferred weights into target model structure.")

    output_ckpt = {
        'epoch': -1,
        'best_fitness': None,
        'model': target_model,
        'ema': None,
        'updates': None,
        'train_args': {}, # Use empty dict for compatibility
        'date': None
}

    # Get the directory part of the output path using os.path
    output_directory = os.path.dirname(output_weights_path)

    # Create the directory recursively, ignoring errors if it exists
    os.makedirs(output_directory, exist_ok=True)
    # --- End Replacement Code ---

    # Save the checkpoint (this line remains unchanged)
    torch.save(output_ckpt, output_weights_path)
    print(f"Saved model with transferred weights to {output_weights_path}")
    print("\nWeight transfer complete.")

else:
    print(f"Found existing transfer weights file: '{output_weights_path}'. Skipping transfer step.")

Transfer weights file '/home/ai-laboratory/Documents/ultralytics-floating-trash/runs/models/yolo11s-dspan.pt' not found. Performing weight transfer...
Loading source weights from /home/ai-laboratory/Documents/ultralytics-floating-trash/yolo11s.pt...
Source weights loaded and state_dict extracted.
Building target model structure from /home/ai-laboratory/Documents/ultralytics-floating-trash/ultralytics/cfg/models/11/yolo11s-dspan.yaml...
Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv

In [ ]:
# --- 3. Fine-Tuning ---
print("\n--- Starting Fine-Tuning ---")

# Load the model with transferred weights
model = YOLO(output_weights_path) # Load the result of the transfer

# Train the model
print(f"Training on data: {data_path}")
results = model.train(
    data=str(data_path),        # Pass path as string
    epochs=150,
    batch=32,
    imgsz=640,
    name='yolo11s-lchbham-flow', # Descriptive name
    # optimizer='AdamW',       # Uncomment to override default
    # momentum=0.9,            # Uncomment to override default
    lr0=0.005,                 # Set initial learning rate
    augment=True,              # Enable augmentation
    # resume=False,            # Set to True to resume training if interrupted
    # device=0,                # Specify GPU device if needed
    # workers=8,               # Adjust based on your system
    # patience=50,             # Early stopping patience
)
print("\n--- Training Finished ---")



--- Starting Fine-Tuning ---
Training on data: /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/AblationDataset/flow/flow.yaml
New https://pypi.org/project/ultralytics/8.3.105 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)
engine/trainer: task=detect, mode=train, model=/home/ai-laboratory/Documents/ultralytics-floating-trash/runs/models/yolo11s-dspan.pt, data=/home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/AblationDataset/flow/flow.yaml, epochs=300, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo11s-lchbham-flow, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, m

train: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/AblationDataset/flow/train/labels.cache... 961 images, 0 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
val: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/AblationDataset/flow/valid/labels.cache... 239 images, 1 backgrounds, 0 corrupt: 100%|██████████| 239/239 [00:00<?, ?it/s]


Plotting labels to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 94 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/31 [00:00<?, ?it/s]/home/ai-laboratory/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/autograd/graph.py:823: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
      1/300      7.71G       2.06      2.943      1.186          4        640: 100%|██████████| 31/31 [00:06<00:00,  4.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.95it/s]

                   all        239        628     0.0151       0.51     0.0107    0.00421


...

  0%|          | 0/31 [00:00<?, ?it/s]/home/ai-laboratory/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/autograd/graph.py:823: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
    300/300      7.97G      0.816     0.3953      0.836          3        640: 100%|██████████| 31/31 [00:05<00:00,  5.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  8.51it/s]

                   all        239        628      0.896      0.866      0.909        0.5



300 epochs completed in 0.542 hours.
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow/weights/last.pt, 19.2MB
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow/weights/best.pt, 19.2MB

Validating /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)
YOLO11s-dspan summary (fused): 120 layers, 9,423,847 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.49it/s]


                   all        239        628      0.891      0.832      0.906       0.49
Speed: 0.1ms preprocess, 3.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow

--- Training Finished ---


In [6]:
# --- 4. Validation ---
print("\n--- Starting Validation ---")

print(f"Validating on flowtest: {test_data_path}")
try:
    res_flow = model.val(data=str(test_data_path)) # Use the trained model object
    print("Validation Results (flowtest):")
    # print(res_flow)
except Exception as e:
    print(f"Error during validation on {test_data_path}: {e}")


print(f"\nValidating on watertrashtest: {test_data_path_wt}")
try:
    res_wt = model.val(data=str(test_data_path_wt)) # Use the trained model object
    print("Validation Results (watertrashtest):")
    # print(res_wt)
except Exception as e:
    print(f"Error during validation on {test_data_path_wt}: {e}")



--- Starting Validation ---
Validating on flowtest: /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/TestDataset/flowtest/flowtest.yaml
Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)


val: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/TestDataset/flowtest/test/labels.cache... 800 images, 1 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:04<00:00,  5.90it/s]


                   all        800       2022      0.816      0.754      0.817      0.405
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow4
Validation Results (flowtest):

Validating on watertrashtest: /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/TestDataset/watertrashtest/watertrashtest.yaml
Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)


val: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/TestDataset/watertrashtest/test/labels.cache... 2800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2800/2800 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 88/88 [00:12<00:00,  7.01it/s]


                   all       2800      10193      0.312     0.0848      0.199     0.0993
Speed: 0.1ms preprocess, 3.5ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-lchbham-flow5
Validation Results (watertrashtest):
